# Preprocess module

## Import Jupyter Module in other files

this code is for importing .ipynb files. YEYEEE

- code form <br/ > 
http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html

In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [2]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [3]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [4]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [5]:
sys.meta_path.append(NotebookFinder())

- word2vec 모듈 다운로드가 요구됨

In [3]:
!pip install gensim
!pip3 install konlpy
!pip install --upgrade jtypes.jpype
!pip3 install JPype1-py3
!pip install xlrd
!pip install https://downloads.sourceforge.net/project/buzhug/buzhug/buzhug-1.8/buzhug-1.8.zip?r=https%3A%2F%2Fsourceforge.net%2Fprojects%2Fbuzhug%2Ffiles%2F&ts=1285445021&use_mirror=switch
!pip install pydblite
!pip install psutil

Requirement already up-to-date: jtypes.jpype in /usr/local/lib/python3.6/site-packages
Requirement already up-to-date: setuptools>=30.4.0 in /usr/local/lib/python3.6/site-packages (from jtypes.jpype)
    100% |████████████████████████████████| 409kB 2.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for psutil ... done
  Stored in directory: /root/.cache/pip/wheels/05/a2/2a/2015d6af91fb5a4cc5bcdfa9699034e2f624fc9cc5acde7ab9
Successfully built psutil


In [4]:
import format_module
import bookingreview
import preprocessreview
import wordvectormaker
import postag_module
import gensim
import xl_to_br_module
#sys.setrecursionlimit(20000)

# 데이터 저장

In [5]:
import _pickle
import gc

def save_object_split(obj, filename, ext, split_size = 1000):
    index = 0
    for i in range(0, len(obj), split_size):
        with open(filename + "_p" + str(index) + "." + ext, 'wb') as output:
            _pickle.dump(obj[i:i+split_size], output)
            index += 1
            mem()
            gc.collect()
                
def save_object(obj, filename):
    with open(filename, 'wb') as output:
        _pickle.dump(obj, output)

def load_object(filename):
    with open(filename, 'rb') as input:
        return _pickle.load(input)
    
def load_object_split(filename, ext, last_index):
    for index in range(last_index+1):
        with open(filename + "_p" + str(index) + "." + ext, 'rb') as input:
            if(index == 0):
                out = _pickle.load(input)
            else:
                out.extend(_pickle.load(input))
    
    return out

# Main

In [6]:
def postagModule(xl_name, reviewDB, mode = "komoran"):
    BRList = xl_to_br_module.xl_to_BookingReview(xl_name)
    
    print("adding review to DB...")
    reviewDB.add_review_list(BRList)
    
    if(mode == "twitter"):
        PRList = postag_module.twitter(BRList)
    else:
        PRList = postag_module.komoran(BRList)
        mode = "komoran"
    '''save_object(PRList_twitter, "save_PR_twitter.pkl")
    save_object(PRList_komoran, "save_PR_komoran.pkl")'''
    
    return PRList

In [7]:
def embeddingModule(PRList):
    word2vec_model = gensim.models.Word2Vec.load('./models/namuwiki_testmodel_Komoran.model')
    print("processing word embedding...")
    return wordvectormaker.word2vec_to_PreprocessReview_list(word2vec_model, PRList)

In [8]:
def formattingModule(PRList, reviewDB, version):
    print("processing formatted review...")
    formatted_list = [format_module.FormattedReview(review) for review in PRList]
    print("saving formatted review...")
    save_object(formatted_list, "./pkl/save_formatted_review_" + version +".pkl")
    return formatted_list

In [9]:
def preprocessModule(xl_name, reviewDB, version):
    PRList = postagModule(xl_name, reviewDB)
    embeddingModule(PRList)
    formatted_list = formattingModule(PRList, reviewDB, version)
    
    return formatted_list

In [13]:
reviewDB = format_module.ReviewDB("./pkl/train_context")
format_module.FormattedReview.setDB(reviewDB)
formatted_list = preprocessModule("./excel/Commonreviews_snuproject_test_context.xlsx", reviewDB, "test_context")

processing 0 th unit...
adding review to DB...
=> Postag initiated
postagging [komoran] 0 ith unit...
processing word embedding...
processing formatted review...
saving formatted review...


In [5]:
#reviewDB = format_module.ReviewDB("validation")
reviewDB = format_module.ReviewDB("train")
print(reviewDB.review_dict[41099].next)

ReviewNode - BookingReview object:
  Company name = 공차 신림역점
  ID = 0___****
  Rating = 10
  Context = 여기 종종 이용하는데 전 항상 친절해서 좋앗는데... 대체적으로 평이 안좋네요  여기 남자분 알바 중에 영웅재중?닮은 키크고 눈 좀 서납게 생기신분 잇는데 항상 갈때마다 음료추천도 잘해주시고 귀찮게 이것저것 물어봐도 웃으시면서 친절하게 응대해주시는게 넘 감사해서 그분 칭찬해드려여... 그 알바분은 회사에서 상 줫으면 좋겟어여 ㅋㅋ신림점 진상손님도 많을텐데 갈때마다 친절해서 항상 기분좋앗어여
  Post time = 41999.687138217596
  Label = False
  Review id = 40093




In [16]:
a = load_object("./pkl/save_formatted_review_train.pkl")
print(a[0])

FormattedReview object 41099:
  context = 
[[ 0.13338269  2.57921386  1.10160124 ...,  0.86103064  2.78854179
   1.87262499]
 [-0.13127854  1.65661263 -0.70222795 ...,  1.08186483  1.34531033
   3.25771809]
 [ 1.09225094  0.87828368  3.05598545 ...,  3.32895207 -0.73563319
  -0.32933012]
 ..., 
 [-4.64829493  5.1486454  -2.75840378 ...,  3.04924154  0.15255213
  -1.48532724]
 [ 1.62699354  1.45267355 -0.51794469 ..., -1.59229398  3.52136612
   1.01794887]
 [ 3.2266624   0.68326694 -0.93881059 ..., -1.524984    1.03987861
  -1.45865238]]
  context_bayes = 
['동네/NNG', '맛집/NNG', '이/VCP', '라/EC', '찾/VV', '았/EP', '는데/EC', '고급/NNG', '스럽/XSA', 'ㄴ/ETM', '한/MM', '정식/NNG', '집/NNG', '에/JKB', '서비스/NNG', '도/JX', '너무/MAG', '친절/NNG', '하/XSV', '아서/EC', '반하/VV', '았/EP', '어요/EC', '가격/NNG', '대비/NNG', '음식/NNG', '도/JX', '푸짐/XR', '하/XSA', '고/EC', '맛있/VA', '어서/EC', '즐겁/VA', 'ㄴ/ETM', '시간/NNG', '보내/VV', '었/EP', '습/NNG', '이/VCP', '니/EC', '당/XSN', '!/SF']
  comp_similarity = 0.5402818532208692
  rate = 1.0
  rei

In [ ]:
print(formatted_list[0].context.shape)
print(formatted_list[1].context.shape)
print(formatted_list[2].context.shape)

In [ ]:
import bookingreview
import format_module

reviewDB = format_module.ReviewDB()
reviewList = [bookingreview.BookingReview("회사", "id", 10, "asdfsadf", 40000.236+i*0.001, True, i) for i in range(50000)]

reviewDB.add_review_list(reviewList)
reviewDB.size()